<a href="https://colab.research.google.com/github/JoaoEmanuel14/trabalho-kaggle-am/blob/main/Trabalho_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho para a Disciplina de Aprendizagem de Máquina

Olá, este Notebook apresenta a resolução da competição do Kaggle 'Toxic Comment Classification Challenge'.

Foi utilizado o modelo de Regressão Logística com abordagem One-vs-Rest para classificação multilabel. A representação textual foi realizada por meio de vetorização TF-IDF considerando unigramas e bigramas, com limite de 100.000 features. O modelo foi avaliado utilizando a métrica ROC-AUC macro, conforme exigido pela competição.

Esse trabalho foi realizado como avaliação para a disciplina de Aprendizagem de Máquina (COMP0424) do Departamento de Computação da Universidade Federal de Sergipe, ministrada pelo Professor Doutor Hendrik Teixeira Macedo.

Os autores do trabalho são João Emanuel Mendonça Apóstolo e José Matheus Ribeiro dos Santos.

# 0. Importando bibliotecas

In [1]:
import os
import zipfile
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

✅ Explicação

os, zipfile, Path → Manipulação de arquivos e diretórios.

pandas → Leitura e manipulação de dados tabulares.

numpy → Operações numéricas.

TfidfVectorizer → Converte texto em vetores numéricos usando TF-IDF.

LogisticRegression → Modelo de classificação linear.

OneVsRestClassifier → Permite usar regressão logística para múltiplas classes independentes.

train_test_split → Divide dados em treino e validação.

roc_auc_score → Métrica de avaliação (usada na competição Kaggle).

# 1. Descompactando Dataset

## Montagem do Google Drive para ter acesso ao Dataset


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Extração do Dataset

In [3]:
# Caminho do dataset
zip_path = '/content/drive/MyDrive/Aprendizagem de Máquina/Trabalho Kaggle/jigsaw-toxic-comment-classification-challenge.zip'

# Caminho para qual iremos extrair
extract_path = Path("/content/jigsaw_data")

# Extrai se o diretório de destino não existir
if not extract_path.exists():
  print(f"Criando diretório {extract_path} e descompactando!")
  os.makedirs(extract_path, exist_ok=True)
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
  print("Dataset extraído com sucesso!")
else:
  print(f"Diretório {extract_path} já existe! Cancelando operação!")

Criando diretório /content/jigsaw_data e descompactando!
Dataset extraído com sucesso!


In [4]:
# Extraindo pastas do dataset
train_zip = '/content/jigsaw_data/train.csv.zip'
test_zip = '/content/jigsaw_data/test.csv.zip'
test_labels_zip = '/content/jigsaw_data/test_labels.csv.zip'

with zipfile.ZipFile(train_zip, 'r') as train_ref:
  train_ref.extractall(extract_path)

with zipfile.ZipFile(test_zip, 'r') as test_ref:
  test_ref.extractall(extract_path)

with zipfile.ZipFile(test_labels_zip, 'r') as test_labels_ref:
  test_labels_ref.extractall(extract_path)

# 2. Observando o Dataset

In [5]:
# Caminhos para o dataset
train_path = '/content/jigsaw_data/train.csv'
test_path = '/content/jigsaw_data/test.csv'
test_labels_path = '/content/jigsaw_data/test_labels.csv'

In [6]:
# Lendo os CSVs
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
test_labels = pd.read_csv(test_labels_path)

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
train['comment_text'][4]

"You, sir, are my hero. Any chance you remember what page that's on?"

In [9]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

(np.float64(394.0732213246768), 590.7202819048923, 5000)

# 3. Leitura do dataset

Separando o X e o y

In [10]:
X = train['comment_text'].fillna("")

y = train[
    ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
]

## Divisão de treino e teste

70% dos dados são voltados para o treino e 30% para validação

O `random_state=42` serve para garantir reprodutibilidade

In [11]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

# 4. Vetorização TF-IDF

Essa vetorização é feita para que o TF-IDF capture padrões linguísticos recorrentes

✅ Explicação dos parâmetros

`max_features=100000` → Limita o vocabulário às 100 mil palavras mais frequentes

`ngram_range=(1,2)` → Usa unigramas e bigramas

`stop_words="english"` → Remove palavras comuns

`strip_accents="unicode"` → Remove acentuação

In [12]:
vectorizer = TfidfVectorizer(
    max_features=100_000,
    ngram_range=(1, 2),
    stop_words="english",
    strip_accents="unicode"
)

In [13]:
X_train_vectorizer = vectorizer.fit_transform(X_train)
X_val_vectorizer = vectorizer.transform(X_val)

# 5. Modelo de Regressão Logística e One-Vs-Rest

- A Regressão Logística é um modelo simples, rápido e produz probabilidades
- O One-Vs-Rest resolve naturalmente o problema *multi-label*

In [14]:
model = OneVsRestClassifier(
    LogisticRegression(
        solver="liblinear",
        max_iter=1000,
        class_weight="balanced"
    )
)

✅ Explicação

- `OneVsRestClassifier`:

  - Treina um classificador para cada classe.

  - Ex: "toxic vs resto", "insult vs resto", etc.

- `solver="liblinear"`:

  - Bom para datasets médios.

- `max_iter=1000`:

  - Aumenta iterações para garantir convergência.

- `class_weight="balanced"`:

  - Corrige desbalanceamento das classes.

## Treinamento

In [15]:
model.fit(X_train_vectorizer, y_train)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=1000,
                                                 solver='liblinear'))

# 6. Avaliação do conjunto de treino

Feita com ROC-AUC

In [16]:
y_val_preds = model.predict_proba(X_val_vectorizer)

roc_auc = roc_auc_score(y_val, y_val_preds, average="macro")

print(f"Score ROC-AUC de treino: {roc_auc:.4f}")

Score ROC-AUC de treino: 0.9777


# 7. Inferência no conjunto de teste

In [17]:
X_test = test['comment_text'].fillna("")

X_test_vectorizer = vectorizer.transform(X_test)

test_preds = model.predict_proba(X_test_vectorizer)

## Criação do arquivo de submissão

In [18]:
submission = pd.DataFrame(
    test_preds,
    columns=[
        "toxic",
        "severe_toxic",
        "obscene",
        "threat",
        "insult",
        "identity_hate"
    ]
)

submission.insert(0, "id", test["id"])
submission.to_csv("submission.csv", index=False)